# Import Dependecies

In [ ]:
try:
    import mesa
    import random
    from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value
    import ipywidgets as widgets
    import pandas as pd
    import matplotlib.pyplot as plt
    import time
    import re
    import time
except:
    !pip install --quiet mesa
    !pip install --quiet pulp
    !pip install --quiet pandas
    !pip install --quiet matplotlib
    !pip install --quiet ipywidgets

# Product

In [ ]:
class Products():
    '''
    This class creates objects for the farms for growing vegetables. this class includes different methods for harvesting,
    calculating different costs, and selling prices.
    '''
    
    '''
    Define the cost per unit of water and electricity.
    These global variables use inside the Farm class methods to find the cost of energy and water.
    '''
    global water_cost 
    water_cost = 0.002 # Euro per liter
    global electricity_cost
    electricity_cost = 0.2 # Euro per kilowatt-hour (kWh)
    
    
    '''
    Here is defined a dictionary icluding required infromation for growing vegetables for each Farm agent.
    '''
    vegetables_dict = {
        "Tomatoes": {"harvest_interval":[60,70,80], # Harvest interval (days) for each quality level
                     "land_required":[1.5, 1.75, 2], # Land required per kg of vegetable (m²/kg) for each quality level
                     "water_required":[200, 225, 250], # Water required per kg of vegetable (liters/kg) for each quality level
                    "energy_required":[0.3,0.35,0.4], # Energy required per kg of vegetable (kWh/kg) for each quality level
                     "other_cost":[0.2, 0.25, 0.3], # Other costs per kg of vegetable (Euro/kg) for each quality level
                     "selling_price":[1,2,3]}, # Selling price per kg of vegetable (Euro/kg) for each quality level
        "Lettuce": {"harvest_interval":[30,37,45], "land_required":[0.5, 0.75, 1], "water_required":[100, 125, 150],
                    "energy_required":[0.1,0.15,0.2], "other_cost":[0.1, 0.15, 0.2], "selling_price":[0.5,1,1.5]},
        "Cucumbers": {"harvest_interval":[50,55,60], "land_required":[1, 1.25, 1.5], "water_required":[200, 225, 250],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2, 0.25, 0.3], "selling_price":[1,1.5,2]},
        "Peppers": {"harvest_interval":[60,67,75], "land_required":[1.5, 1.75, 2], "water_required":[250, 275, 300],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2, 0.25, 0.3], "selling_price":[1,1.75,2.5]},
        "Eggplants": {"harvest_interval":[60,65,70], "land_required":[1.5, 1.75, 2], "water_required":[200, 225, 250],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2, 0.25, 0.3], "selling_price":[1,1.5,2]},
        "Basil": {"harvest_interval":[45,52,60], "land_required":[0.5, 0.75, 1], "water_required":[100, 125, 150],
                    "energy_required":[0.1,0.15,0.2], "other_cost":[0.1,0.15,0.2], "selling_price":[0.5,0.75,1]},
        "Arugula": {"harvest_interval":[30,37,45], "land_required":[0.5, 0.75, 1], "water_required":[100, 125, 150],
                    "energy_required":[0.1,0.15,0.2], "other_cost":[0.1,0.15,0.2], "selling_price":[0.5,1,1.5]},
        "Spinach": {"harvest_interval":[30,37,45], "land_required":[0.5, 0.75, 1], "water_required":[100, 125, 150],
                    "energy_required":[0.1,0.15,0.2], "other_cost":[0.1,0.15,0.2], "selling_price":[0.5,1,1.5]},
        "Radishes": {"harvest_interval":[25,27,35], "land_required":[0.5, 0.75, 1], "water_required":[100, 125, 150],
                    "energy_required":[0.1,0.15,0.2], "other_cost":[0.1,0.15,0.2], "selling_price":[0.3,0.55,0.8]},
        "Carrots": {"harvest_interval":[60,65,70], "land_required":[1.5, 1.75, 2], "water_required":[200, 225, 250],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2,0.25,0.3], "selling_price":[1,1.5,2]},
        "Onions": {"harvest_interval":[110,120,130], "land_required":[1, 1.25, 1.5], "water_required":[200, 225, 250],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2,0.25,0.3], "selling_price":[1,1.5,2]},
        "Potatoes": {"harvest_interval":[90,105,120], "land_required":[1, 1.25, 1.5], "water_required":[200, 225, 250],
                    "energy_required":[0.3,0.35,0.4], "other_cost":[0.2,0.25,0.3], "selling_price":[1,1.5,2]}
    }
    
    def __init__(self, quality: str, farming_method: list, vegetables: list, land: int):
        """
        Initialize the `Products` object with the specified quality, farming method vegetables to be cultivated, and land area.

        Args:
            quality (str): The quality level of the vegetables ('Low', 'Average', or 'High')
            vegetables (str): A list of vegetables to be cultivated (e.g., ['Tomatoes', 'Lettuce'])
            land (int): The total land area available for cultivation (default: 1000 m²)
        """
        self.land = land
        self.vegetables = vegetables
        self.quality = quality
        self.farming_method = farming_method

    def get_index(self) -> dict:
        """
        Get the index of the specified quality in the `vegetables_dict`.

        - For Price-Based Strategy farm is the index of 'Low' quality considered.
        - For Quality-Based Strategy farm is the index of 'High' quality considered.
        - For Eco-Environmentally Focused Strategy farm is the index of 'Average' quality considered.

        Raises:
            ValueError: If the quality is not defined in the `vegetables_dict`.
        """
        qualities = ['Low', 'Average', 'High']
        index_map = {quality: index for index, quality in enumerate(qualities)}
        try:
            return index_map[self.quality]
        except KeyError:
            raise ValueError(f"Quality {self.quality} is not defined correctly.")

    def Cultivate(self) -> dict:
        """
        Calculate the weekly production of each vegetable based on the specified quality and land area.
        It assumes that the farm has the growing planning in the way to have specific amount of all growing vegetables
        every week to sell in the market.

        Returns:
            dict: A dictionary containing the weekly production of each vegetable (kg/week)
        """
        index = self.get_index()
        number_of_vegetables = len(self.vegetables)
        devided_land = self.land / number_of_vegetables
        production_dict = dict()
        for veg in self.vegetables:
            # Get the harvest interval and annual production cycles for the specified vegetable
            harvest_interval = Products.vegetables_dict[veg]['harvest_interval'][index] # days
            production_cycles_per_year = 365 // harvest_interval # number
            
            # Get the land required and weekly production for the specified vegetable
            land_required = self.vegetables_dict[veg]['land_required'][index] # m²/kg
            weekly_production = ((devided_land / land_required) * production_cycles_per_year) / 52
            
            # Round the weekly production to one decimal place and add it to the dictionary
            production_dict[veg] = round(weekly_production, 1)
        return production_dict
    
    def calculate_water(self) -> dict:
        """
        Calculate the weekly water consumption for each vegetable based on the specified quality and water requirements.

        Returns:
            dict: A dictionary containing the weekly water consumption of each vegetable (liters/week)
        """
        water_required_dict = self.Cultivate()
        index = self.get_index()
        
        for veg in self.vegetables:
            # Get the water required per kg and calculate the weekly water consumption
            water_per_kg = self.vegetables_dict[veg]['water_required'][index] # liter/kg
            # Update the dictionary with the weekly water consumption
            water_required_dict[veg] *= round(water_per_kg, 1)
        return water_required_dict
    
    def calculate_energy(self) -> dict:
        """
        Calculate the weekly energy consumption for each vegetable based on the specified quality and energy requirements.

        Returns:
            dict: A dictionary containing the weekly energy consumption of each vegetable (kWh/week)
        """
        energy_required_dict = self.Cultivate()
        index = self.get_index()
        
        for veg in self.vegetables:
            # Get the energy required per kg and calculate the weekly energy consumption
            energy_required = self.vegetables_dict[veg]['energy_required'][index] # KWh/kg
            energy_required_dict[veg] *= round(energy_required, 3) # kWh/week
        return energy_required_dict
    
    def calculate_other_cost(self) -> dict:
        """
        Calculate the weekly other costs for each vegetable based on the specified quality and other cost requirements.

        Returns:
            dict: A dictionary containing the weekly other costs of each vegetable (Euro/week)
        """
        other_cost_dict = self.Cultivate()
        index = self.get_index()
        
        for veg in self.vegetables:
            # Get the other cost per kg and calculate the weekly other cost
            other_cost = self.vegetables_dict[veg]['other_cost'][index] 
            other_cost_dict[veg] *= other_cost 
        return other_cost_dict
    
    def calculate_production_cost(self) -> dict:
        """
        Calculate the weekly production cost for each vegetable based on the specified quality, water cost, electricity cost, 
        and other costs.

        Returns:
            dict: A dictionary containing the weekly production cost of each vegetable (Euro/week)
        """
        index = self.get_index()
        production_cost_dict = dict()
        
        for veg in self.vegetables:
            # Get the water cost, energy cost, and other cost per kg
            water_cost_per_kg = Products.vegetables_dict[veg]['water_required'][index] * Products.water_cost
            energy_cost_per_kg = Products.vegetables_dict[veg]['energy_required'][index] * Products.electricity_cost
            other_cost = Products.vegetables_dict[veg]['other_cost'][index]
            
            # Calculate the total production cost per kg
            production_cost_dict.update({veg:round(water_cost_per_kg + energy_cost_per_kg + other_cost, 2)}) # Euro/kg
        return production_cost_dict
    
    def sell_price(self) -> dict:
        '''
        This method return back the selling price of each vegetable based on the quality given in the 'vegetables_dict'
        '''
        selling_price_dict = self.Cultivate()
        index = self.get_index()
        
        for veg in self.vegetables:
            selling_price = self.vegetables_dict[veg]['selling_price'][index]
            selling_price_dict.update({veg:selling_price})
        return selling_price_dict
    
    def harvest(self) -> dict:
        '''
        This methods return back a dictionary for all the products of the specified farm object including its farming method,
        quality and price
        '''
        harvest = dict()
        for veg, value in self.Cultivate().items():
            harvest[veg] = {"Amount":value}
        for veg, value in self.sell_price().items():
            harvest[veg].update({"Price":value})
            harvest[veg].update({"Quality":self.quality})
            harvest[veg].update({"Farming Method":self.farming_method})
        return harvest

<h1>Agents Class: Customer</h1>

In [ ]:
class Customer(mesa.Agent):
    '''
    this class has differnt methods for Customer agent.
    this class inherits from the mesa package for creating the agent.
    '''
    def __init__(self, unique_id: int, model, pos: tuple, farms_indexes: list, budget_min: float, budget_max: float):
        super().__init__(unique_id, model)
        # this position value is used for defining the position of agents:
        self.pos = pos
        # here is initiated the all farm rates by 0 when creating the new customer agents.
        self.farms_rates = dict.fromkeys(farms_indexes, 0)
        # the related varibles for min and max budget of customer
        self.min = budget_min
        self.max = budget_max

    def set_budget(self) -> float:
        # create a random budget for customer:
        return round(random.uniform(self.min, self.max), 1)
    
    def set_basket(self) -> dict:
        # Initiate the random amount of reqired vegetables for the customer:
        return dict(Tomatoes = random.choice([0, 0.5, 0.75, 1]),
                        Potatoes = random.choice([0, 0.5, 0.75, 1]),
                        Carrots = random.choice([0, 0.5, 0.75, 1]))
    
    def select_farm(self) -> dict:
        '''
        This method returns back a dictionary of farm rating for each customer based on their experience whith each farm.
        it will sort the dictionary from highest to lowest rate for each farm. 
        it help to customer to move at the first to the farm with highest rate.
        '''
        keys = list(self.farms_rates.keys())
        random.shuffle(keys)
        shuffled_dict = dict()
        for key in keys:
            shuffled_dict.update({key: self.farms_rates[key]})
        sorted_tuple = sorted(shuffled_dict.items(), key=lambda x:x[1], reverse=True)
        self.farms_rates = dict(sorted_tuple)
        return self.farms_rates
                                                                         
    def buy_products(self):
        '''
        in this method the purchase is doing, and move number, customer satisfaction and also farm rating is conducting.
        '''
        # farm_indexes is an object from the select_farm method 
        for farm_index in self.farm_indexes:
            for farm in self.farms_list:
                # Check if the customer has a positive budget
                if not self.customer_budget > 0:
                    break
                if all(value == 0 for value in self.need_products.values()):
                    break
                # Identify the farm based on its unique ID
                if farm.unique_id != farm_index:
                    continue
                # customer has a move to the farm:
                self.move += 1
                # Check if the customer meets the farm's conditions
                if not self.check_condition(farm):
                    continue
                # Purchase the required vegetables based on the customer's needs
                self.optimize_purchase(farm)
                # Check if the customer has bought all required vegetables and still has a budget
                if not any(self.need_products.values()) and self.customer_budget > 0:
                    # when customer has bought all of their needs as well as additional vegetables then give value to 
                    # its rate on the farm_rates dictinary:
                    self.farms_rates[farm_index] += 1
                    # add a value to the customer satisfaction
                    self.satisfaction += 1
                    # Purchase additional vegetables if any are available
                    self.optimize_additional_purchase(farm)
                    # if the cutomer bought all required vegetables and either additional, then break the loop.
                    break
                else:
                    # when customer has bought part of their needs vegetables or all of them and doesn't have more budget:
                    self.farms_rates[farm_index] += 0.5
                    self.satisfaction += 0.5
                        
                        
    def check_condition(self, farm) -> bool:
        '''Each customer has differnt condition, and they defined inside the related classes (children classes).'''
        pass
                                     
    def optimize_purchase(self, farm): 
        '''
        Here an optimization algorithm for purchasing is defined. each customer based on their vegetable needs and budget
        doing purchase.
        '''
        # this dictionary has a condition to find the min value from required vegetables of customer and available vegetables
        # inside the farm
        min_values = {veg: min(values['Amount'], self.need_products[veg])
                 for veg, values in farm.products.items() if veg in self.need_products}
        
        # Create the optimization problem
        model = LpProblem('VegetablePurchase', LpMaximize)

        # Define the decision variables
        variables = LpVariable.dicts('quantity', min_values.keys(), lowBound=0, cat='Continuous')

        # Define the objective function
        model += lpSum([variables[veg] for veg in min_values.keys()])

        # Define the budget constraint
        model += lpSum([variables[veg] * farm.products[veg]['Price'] for veg in min_values.keys()]) <= self.customer_budget

        # Define the quantity constraints for each vegetable
        for veg in min_values.keys():
            model += variables[veg] <= min_values[veg]

        # Solve the optimization problem
        model.solve() 
        
        # check if the optimization model has the optimal solution:
        if model.status == 1:
            for veg in min_values.keys():
                # add purchased products to the customer basket:
                self.customer_basket[veg] += value(variables[veg])
                # remove the amount of purchased products from the need_products dictionary of the customer:
                self.need_products[veg] -= value(variables[veg])
                # subtract the cost of purchase from customer's budget:
                self.customer_budget -= round(value(variables[veg]) * farm.products[veg]['Price'],1)
                # subtract the amount of purchased products from farm's products:
                farm.products[veg]['Amount'] -= value(variables[veg])
                # increase the cost of purchase to the farm profit variable
                farm.farm_profit += round(value(variables[veg]) * farm.products[veg]['Price'],1)
        else:
            print("No feasible solution found for optimize_purchase.")
                
    def optimize_additional_purchase(self, farm):
        # Check if the farm has more different available type of vegetable:
        if any(veg not in self.need_products and values['Amount'] != 0 for veg, values in farm.products.items()):
            
            # this dictionary has a condition to find the max amount of products availabe in the farm that isn't
            # inside the customer needs vegetables:
            max_values = {veg: farm.products[veg]['Amount'] for veg, values in farm.products.items() 
              if veg not in self.need_products and farm.products[veg]['Amount'] != 0}
 
            # Create the optimization problem
            model = LpProblem('AdditionalVegetablePurchase', LpMaximize)

            # Define the decision variables
            variables = LpVariable.dicts('quantity', max_values.keys(), lowBound=0, cat='Continuous')

            # Define the objective function
            model += lpSum([variables[veg] for veg in max_values.keys()])

            # Define the budget constraint
            model += lpSum([variables[veg] * farm.products[veg]['Price'] for veg in max_values.keys()]) <= self.customer_budget

            # Define the quantity constraints for each vegetable
            for veg in max_values.keys():
                model += variables[veg] <= max_values[veg]

            # Solve the optimization problem
            model.solve()
            
            if model.status == 1:
                for veg in max_values.keys():
                    self.customer_basket[veg] = value(variables[veg])
                    self.customer_budget -= round(value(variables[veg]) * farm.products[veg]['Price'],1)
                    farm.products[veg]['Amount'] -= value(variables[veg])
                    farm.farm_profit += round(value(variables[veg]) * farm.products[veg]['Price'],1)
            else:
                print("No feasible solution found for optimize_additional_purchase.")
    
    def step(self, farms_list: list):
        '''
        Here in this method all of the process that should be done in each step is defined.
        '''
        # create a random budget for the customer:
        self.customer_budget = self.set_budget()
        # create the need products for the customer, and call it first_need_products in order to not change when the customer
        # buu products. becase this variable is used in collecting data:
        self.first_need_products = self.set_basket()
        # create a copy of first_need_products and call it need_products to use for buying products:
        self.need_products = self.first_need_products.copy()
        # create an empty customer basket for the cutomer from their need products, to add bought vegetables from farms:
        self.customer_basket = {veg: 0 for veg in self.need_products.keys()}
        # create farm_indexes variable based on the farm rating that in previous steps has done for each farm based on the
        # customer experiences
        self.farm_indexes = self.select_farm()
        # take the list of the available farms:
        self.farms_list = farms_list
        # initiate customer satisfaction
        self.satisfaction = 0
        # initiate customer number move to farms
        self.move = 0
        # cutomer satisfction is defined based on the purchase experience and number of move:
        self.buy_products()
        if self.satisfaction != 0:
            self.satisfaction /= self.move
        else:
            self.satisfaction = 0

    
        
class Customer_Price_Sensitive(Customer):
    '''
    Price-Sensitive Customer:
    '''
    def __init__(self, unique_id, model, pos, farms_indexes, budget_min, budget_max):
        super().__init__(unique_id, model, pos, farms_indexes, budget_min, budget_max)
        
    def set_budget(self):
        return super().set_budget()
    
    def set_basket(self):
        '''
        The required vegetables are the same that defined in the parent class.
        '''
        return super().set_basket()
    
    def select_farm(self):
        return super().select_farm()
    
    def buy_products(self):
        return super().buy_products()
    
    def check_condition(self, farm) -> bool:
        # Check if any customer's required vegetable is available in the farm:
        if any(value > 0 and farm.products[veg]['Amount'] > 0 for veg, value in self.need_products.items()):
            return True
                                
    def optimize_purchase(self, farm):
        return super().optimize_purchase(farm)
    
    def optimize_additional_purchase(self, farm):
        return super().optimize_additional_purchase(farm)
    
    def step(self, farms_list):
        super().step(farms_list)
           

class Customer_Quality_Oriented(Customer):
    '''
    Quality-Oriented Customer:
    '''
    def __init__(self, unique_id, model, pos, farms_indexes, budget_min, budget_max):
        super().__init__(unique_id, model, pos, farms_indexes, budget_min, budget_max)
             
    def set_budget(self):
        return super().set_budget()
    
    def set_basket(self):
        '''
        The required vegetables are the same that defined in the parent class.
        '''
        return super().set_basket()
    
    def select_farm(self):
        return super().select_farm()
    
    def buy_products(self):
        return super().buy_products()
                                   
    def check_condition(self, farm) -> bool:
        for value in farm.products.values():
            # Check if the vegetables quality are high or average:
            if value["Quality"] == 'High' or value["Quality"] == 'Average':
                # Check if any customer's required vegetable is available in the farm:
                if any(value > 0 and farm.products[veg]['Amount'] > 0 for veg, value in self.need_products.items()):
                    return True
            
    def optimize_purchase(self, farm):
        return super().optimize_purchase(farm)

    def optimize_additional_purchase(self, farm):
        return super().optimize_additional_purchase(farm)
                
    def step(self, farms_list):
        super().step(farms_list)
        
class Customer_Environmental_Conscious(Customer):
    '''
    Environmentally Conscious Customer:
    '''
    def __init__(self, unique_id, model, pos, farms_indexes, budget_min, budget_max):
        super().__init__(unique_id, model, pos, farms_indexes, budget_min, budget_max)
        
    def set_budget(self):
        return super().set_budget()
    
    def set_basket(self):
        '''
        The required vegetables are the same that defined in the parent class.
        '''
        return super().set_basket()
    
    def select_farm(self):
        return super().select_farm()
    
    def buy_products(self):
        return super().buy_products()
    
    def check_condition(self, farm):
        for value in farm.products.values():
            # Check if the farming method of vegetables in farm are sustainable:
            if 'sustainable' in value['Farming Method']:
                # Check if any customer's required vegetable is available in the farm:
                if any(value > 0 and farm.products[veg]['Amount'] > 0 for veg, value in self.need_products.items()):
                    return True
                                
    def optimize_purchase(self, farm):
        return super().optimize_purchase(farm)
    
    def optimize_additional_purchase(self, farm):
        return super().optimize_additional_purchase(farm)
    
    def step(self, farms_list):
        super().step(farms_list)

    
class Customer_Dietary_Preference(Customer):
    '''
    Dietary Preference-Based Customer:
    '''
    def __init__(self, unique_id, model, pos, farms_indexes, budget_min, budget_max):
        super().__init__(unique_id, model, pos, farms_indexes, budget_min, budget_max)
        
    def set_budget(self):
        return super().set_budget()
    
    def set_basket(self):
        '''
        The type of required vegetables are defined here and their amount are defined randomly:
        '''
        return dict(Tomatoes = random.choice([0, 0.5, 0.75, 1]),
                        Potatoes = random.choice([0, 0.5, 0.75, 1]),
                        Carrots = random.choice([0, 0.5, 0.75, 1]),
                       Eggplants = random.choice([0, 0.5, 0.75, 1]),
                       Spinach = random.choice([0, 0.5, 0.75, 1]))
    
    def select_farm(self):
        return super().select_farm()
    
    def buy_products(self):
        return super().buy_products()
    
    def check_condition(self, farm) -> bool:
        for value in farm.products.values():
            # Check if the farming method in the farm is organic:
            if 'organic' in value['Farming Method']:
                # Check if any customer's required vegetable is available in the farm:
                if any(value > 0 and farm.products[veg]['Amount'] > 0 for veg, value in self.need_products.items()):
                    return True
 
    def optimize_purchase(self, farm):
        return super().optimize_purchase(farm)
    
    def optimize_additional_purchase(self, farm):
        return super().optimize_additional_purchase(farm)
    
    def step(self, farms_list):
        super().step(farms_list)
        

<h1>Agents Class: Farm</h1>

In [ ]:
class Farm(mesa.Agent):
    '''
    this class has differnt methods for Farm agent.
    this class inherits from the mesa package for creating the agent.
    '''
    def __init__(self, unique_id: int, model, pos: tuple, farm_profit: float, area: int):
        super().__init__(unique_id, model)
        # farm position:
        self.pos = pos
        # famr profit:
        self.farm_profit = farm_profit
        # farm area:
        self.area = area

    def harvest_vegetables(self, quality: str, farming_method: list, vegetables: list) -> dict:
        self.quality = quality
        self.farming_method = farming_method
        self.vegetables = vegetables
        return Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).harvest()
    
    def buy_water(self) -> float:
        water_dict = Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).calculate_water()
        # calculate the sum of total required water for one week per liter, multiply to the price of water per liter:
        return sum(water_dict.values()) * water_cost
    
    def buy_electricity(self) -> float:
        energy_dict = Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).calculate_energy()
        # calculate the sum of total required energy for one week per kilowatt-hour (kWh), multiply to the price of electricity:
        return sum(energy_dict.values()) * electricity_cost
    
    def calculate_other_cost(self) -> float:
        other_cost_dict = Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).calculate_other_cost()
        return sum(other_cost_dict.values())

    def step(self):
        '''
        in the method, all the procces that should be done in each step is defined.
        '''
        # define the weekly porducts of the farm:
        self.products = self.harvest_vegetables()
        # pay the water cost for previous step (week) and subtract from farm profit:
        self.farm_profit -= self.buy_water()
        # pay the electricity cost for previous step (week) and subtract from farm profit:
        self.farm_profit -= self.buy_electricity()
        # pay the other cost for previous step (week) and subtract from farm profit:
        self.farm_profit -= self.calculate_other_cost()
        
    
class Farm_Price_Based(Farm):
    '''
    Farm Price Based Strategy
    '''
    def __init__(self, unique_id, model, pos, farm_profit, area):
        super().__init__(unique_id, model, pos, farm_profit, area)
        
    def harvest_vegetables(self, quality="Low", farming_method=["non-organic"],
                           vegetables=["Tomatoes", "Potatoes", "Carrots"]):        
        return super().harvest_vegetables(quality, farming_method, vegetables)
    
    def buy_water(self):
        return super().buy_water()
    
    def buy_electricity(self):
        return super().buy_electricity()
    
    def calculate_other_cost(self):
        return super().calculate_other_cost()

    def step(self):
        return super().step()

class Farm_Quality_Based(Farm):
    '''
    Farm Quality Based Strategy:
    '''
    def __init__(self, unique_id, model, pos, farm_profit, area):
        super().__init__(unique_id, model, pos, farm_profit, area)

    def harvest_vegetables(self, quality="High", farming_method=["organic"], 
                           vegetables=["Tomatoes", "Potatoes", "Carrots", "Eggplants", "Spinach"]):
        return super().harvest_vegetables(quality, farming_method, vegetables)

    def buy_water(self):
        return super().buy_water()
    
    def buy_electricity(self):
        return super().buy_electricity()
    
    def calculate_other_cost(self):
        return super().calculate_other_cost()

    def step(self):
        return super().step()     
        
class Farm_Sustainable_Organic_Based(Farm):
    '''
    Farm Eco-Environmentally Focused Strategy:
    '''
    def __init__(self, unique_id, model, pos, farm_profit, area):
        # for this farm we should define the water and energy variable because this farm collect rain water and also generate
        # renewable energy:
        self.water = 0
        self.energy = 0
        super().__init__(unique_id, model, pos, farm_profit, area)

    def harvest_vegetables(self, quality="Average", farming_method=["organic", "sustainable"], 
                           vegetables=["Tomatoes", "Potatoes", "Carrots", "Eggplants", "Spinach"]):
        return super().harvest_vegetables(quality, farming_method, vegetables)

    def buy_water(self):
        # Calculate the amount of required water for each week:
        water_dict = Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).calculate_water()
        # Calculate the potential amount of rainwater collected (liters per week):
        self.water += collect_rain_water(self.step_number, self.area)
        
        if self.water >= sum(water_dict.values()):
            self.water -= sum(water_dict.values())
            buy_water = 0
        else:
            buy_water = (sum(water_dict.values()) - self.water) * water_cost
            self.water = 0
        return buy_water
    
    def buy_electricity(self):
        # Calculate the amount of required energy for each week:
        energy_dict = Products(quality=self.quality, farming_method=self.farming_method, vegetables=self.vegetables, 
                            land=self.area).calculate_energy()
        
        self.energy += generate_solar_energy(self.step_number) # kWh/week
        
        if self.energy >= sum(energy_dict.values()):
            self.energy -= sum(energy_dict.values())
            buy_energy = 0
        else:
        # calculate the sum of total required energy for one week per kilowatt-hour (kWh), 
        # multiply to the price of electricity:
            buy_energy = (sum(energy_dict.values()) - self.energy) * electricity_cost
            self.energy = 0
        return buy_energy
    
    def calculate_other_cost(self):
        return super().calculate_other_cost()

    def step(self, step_number: int):
        # to calculate the amount of generated renewable energy and rain water collected for this farm class, we need
        # to pass the number of step to the related functions:
        self.step_number = step_number
        return super().step()   
    

# Helper Functions

In [ ]:
def collect_rain_water(step_number: int, area: int) -> float:
    weekly_rainfall = pd.read_csv('weekly_rainfall.csv')
        
    # Calculate the catchment area of the farm
    global catchment_area
    catchment_area = area  # square meters
    # Calculate the average runoff coefficient for the catchment area
    global runoff_coefficient
    runoff_coefficient = 0.6  # dimensionles
    # Calculate the depth of the rainwater harvesting system
    global depth 
    depth = 0.5  # meters
    # Calculate the potential amount of rainwater collected (liters per week)
    return weekly_rainfall['precip'][step_number] * catchment_area * runoff_coefficient * depth

def generate_solar_energy(step_number: int) -> float:
    weekly_clearsky_GHI = pd.read_csv('weekly_global_horizontal_irradiance.csv')
    
    global pv_price
    pv_price = 274 # Euro
    #pv_link = 'https://www.europe-solarstore.com/lg-neon-2-bifacial-lg415n2t-l5.html'
    global pv_model
    pv_model = 'PV LG Neon 2 Bifacial LG415N2T L5'
    global eff
    eff = 0.20
    lenght = 2024/1000 #m
    width = 1024/1000 #m
    global A
    A = lenght * width
    Tref = 0.35 / 100
    global PV_number
    PV_number = 1
    
    # renewable energy from PV for each week based on the step or week number (kWh/week):    
    return eff * PV_number*A*weekly_clearsky_GHI['Clearsky GHI'][step_number]*\
                                                        (1-Tref*weekly_clearsky_GHI['Temperature'][step_number])/1000 

# Functions for collecting data

In [ ]:
def get_average_move_data(model) -> dict:
    move_average_data = {
        'Price Sensitive_average_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Price_Sensitive)) / model.n_cps,
        'Quality Oriented_average_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Quality_Oriented)) / model.n_cqo,
        'Environmentally Conscious_average_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Environmental_Conscious)) / model.n_cec,
        'Dietary Preference-Based_average_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Dietary_Preference)) / model.n_cdp
    }
    return move_average_data

def ge_num_move_data(model) -> dict:
    move_data = {
        'Price Sensitive_num_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Price_Sensitive)),
        'Quality Oriented_num_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Quality_Oriented)),
        'Environmentally Conscious_num_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Environmental_Conscious)),
        'Dietary Preference-Based_num_move': sum(agent.move for agent in model.schedule.agents if isinstance
                   (agent, Customer_Dietary_Preference))
    }
    return move_data

def get_satisfaction_data(model) -> dict:
    satisfaction_data = {
        'Price Sensitive_satisfaction': round(sum(agent.satisfaction for agent in model.schedule.agents if isinstance
                   (agent, Customer_Price_Sensitive)) / model.n_cps,2) * 100,
        'Quality Oriented_satisfaction': round(sum(agent.satisfaction for agent in model.schedule.agents if isinstance
                   (agent, Customer_Quality_Oriented)) / model.n_cqo,2) * 100,
        'Environmentally Conscious_satisfaction': round(sum(agent.satisfaction for agent in model.schedule.agents if isinstance
                   (agent, Customer_Environmental_Conscious)) / model.n_cec,2) * 100,
        'Dietary Preference-Based_satisfaction': round(sum(agent.satisfaction for agent in model.schedule.agents if isinstance
                   (agent, Customer_Dietary_Preference)) / model.n_cdp,2) * 100
    }
    return satisfaction_data

def get_average_weight_need_products_data(model) -> dict:
    weight_need_products_data = {
        'Price Sensitive_average_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                for agent in model.schedule.agents if isinstance(agent, Customer_Price_Sensitive)) / model.n_cps,2),
        'Quality Oriented_average_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                for agent in model.schedule.agents if isinstance(agent, Customer_Quality_Oriented)) / model.n_cqo,2),
        'Environmentally Conscious_average_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                for agent in model.schedule.agents if isinstance(agent, Customer_Environmental_Conscious)) / model.n_cec,2),
        'Dietary Preference-Based_average_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                for agent in model.schedule.agents if isinstance(agent, Customer_Dietary_Preference)) / model.n_cdp,2)
    }
    return weight_need_products_data

def get_weight_need_products_data(model) -> dict:
    weight_need_products_data = {
        'Price Sensitive_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Price_Sensitive)),2),
        'Quality Oriented_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Quality_Oriented)),2),
        'Environmentally Conscious_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Environmental_Conscious)),2),
        'Dietary Preference-Based_weight_need': round(sum(sum(value for value in agent.first_need_products.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Dietary_Preference)),2)
    }
    return weight_need_products_data

def get_average_weight_bought_products_data(model) -> dict:
    weight_bought_products_data = {
        'Price Sensitive_average_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Price_Sensitive)) / model.n_cps,2),
        'Quality Oriented_average_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Quality_Oriented)) / model.n_cqo,2),
        'Environmentally Conscious_average_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) 
                        for agent in model.schedule.agents if isinstance(agent, Customer_Environmental_Conscious)) / model.n_cec,2),
        'Dietary Preference-Based_average_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) 
                    for agent in model.schedule.agents if isinstance(agent, Customer_Dietary_Preference)) / model.n_cdp,2)
    }
    return weight_bought_products_data

def get_weight_bought_products_data(model) -> dict:
    weight_bought_products_data = {
        'Price Sensitive_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) for agent in model.schedule.agents 
                         if isinstance(agent, Customer_Price_Sensitive)),2),
        'Quality Oriented_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) for agent in model.schedule.agents
                         if isinstance(agent, Customer_Quality_Oriented)),2),
        'Environmentally Conscious_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) for agent in model.schedule.agents 
                         if isinstance(agent, Customer_Environmental_Conscious)),2),
        'Dietary Preference-Based_weight_bought': round(sum(sum(value for value in agent.customer_basket.values()) for agent in model.schedule.agents
                         if isinstance(agent, Customer_Dietary_Preference)),2)
    }
    return weight_bought_products_data

def get_farm_budget_data(model) -> dict:
    farm_budget_data = {
        'Price Based_budget': round(sum(agent.farm_profit for agent in model.schedule.agents 
                                                                 if isinstance(agent, Farm_Price_Based)),2),
        'Quality Based_budget': round(sum(agent.farm_profit for agent in model.schedule.agents 
                                                                 if isinstance(agent, Farm_Quality_Based)),2),
        'Eco-Environmentally Focused_budget': round(sum(agent.farm_profit for agent in model.schedule.agents 
                                                                  if isinstance(agent, Farm_Sustainable_Organic_Based)),2)
    }
    return farm_budget_data

def get_farm_waste_data(model) -> dict:       
    farm_waste_data = {
    "Price Based_waste": round(sum([val['Amount'] for agent in model.schedule.agents if isinstance(agent, Farm_Price_Based)
                      for val in agent.products.values()]),2),
    "Quality Based_waste": round(sum([val['Amount'] for agent in model.schedule.agents if isinstance(agent, Farm_Quality_Based)
                      for val in agent.products.values()]),2),
    "Eco-Environmentally Focused_waste": round(sum([val['Amount'] for agent in model.schedule.agents if isinstance(agent, Farm_Sustainable_Organic_Based)
                      for val in agent.products.values()]),2)
    }
    return farm_waste_data

<h1>Model Class</h1>

In [ ]:
class FEW(mesa.Model):
    '''
    Manger class to run the Farms and Customers agents.
    This class inherits from the mesa Modle.
    '''
    def __init__(self, 
                 num_customer_price_sensitive: int, 
                 num_customer_quality_oriented: int, 
                 num_Customer_Environmental_Conscious: int,
                 num_Customer_Dietary_Preference: int, 
                 budget_min_customer_price_sensitive: float, 
                 budget_max_customer_price_sensitive: float,
                 budget_min_customer_quality_oriented: float,
                 budget_max_customer_quality_oriented: float,
                 budget_min_Customer_Environmental_Conscious: float, 
                 budget_max_Customer_Environmental_Conscious: float,
                 budget_min_Customer_Dietary_Preference: float, 
                 budget_max_Customer_Dietary_Preference: float,
                 num_Farm_Price_Based: int, 
                 num_Farm_Quality_Based: int, 
                 num_Farm_Sustainable_Organic_Based: int,
                 area_Farm_Price_Based: int,
                 area_Farm_Quality_Based: int,
                 area_Farm_Sustainable_Organic_Based: int,
                 step_count: int,
                 width: int, 
                 height: int):

        self.width = width
        self.height = height 
        self.n_cps = num_customer_price_sensitive
        self.n_cqo = num_customer_quality_oriented
        self.n_cec = num_Customer_Environmental_Conscious
        self.n_cdp = num_Customer_Dietary_Preference
        self.budget_min_cps = budget_min_customer_price_sensitive
        self.budget_max_cps = budget_max_customer_price_sensitive
        self.budget_min_cqo = budget_min_customer_quality_oriented
        self.budget_max_cqo = budget_max_customer_quality_oriented
        self.budget_min_cec = budget_min_Customer_Environmental_Conscious
        self.budget_max_cec = budget_max_Customer_Environmental_Conscious
        self.budget_min_cdp = budget_min_Customer_Dietary_Preference
        self.budget_max_cdp = budget_max_Customer_Dietary_Preference
        self.n_fpb = num_Farm_Price_Based
        self.n_fqb = num_Farm_Quality_Based
        self.n_fsob = num_Farm_Sustainable_Organic_Based
        self.area_fpb = area_Farm_Price_Based
        self.area_fqb = area_Farm_Quality_Based
        self.area_fsob = area_Farm_Sustainable_Organic_Based
        # Initiate farm profits
        self.farm_profit_fpb = 0
        self.farm_profit_fqb = 0
        self.farm_profit_fsob = 0
        self.step_count = step_count
        
        # initiate the step number to count:
        self.step_number = 0 
        '''
        The scheduler controls the order in which agents are activated, causing the agent to take their defined action.
        The scheduler is also responsible for advancing the model by one step.
        '''
        # Create scheduler and assign it to the model:
        self.schedule = mesa.time.RandomActivationByType(self)
        
        # instantiate a grid with width and height parameters, and a false boolean as to whether the grid is not toroidal:
        #self.grid = mesa.space.SingleGrid(self.width, self.height, torus=False)
        
        # Initiate an empty list for farm_indexes (for farm rating for step method in Customer class):
        farms_indexes = list()
        
        # Create the agents's attribute (agent_id, unique_id) and set the initial values:
        agent_id = 0
        
        # Farm Agent : Farm_Price_Based (fpb)
        for n in range(self.n_fpb):
            # Define the agent's positions randomly:
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_fpb = Farm_Price_Based(agent_id, self, (x,y), self.farm_profit_fpb, self.area_fpb)
            farms_indexes.append(a_fpb.unique_id)
            # add the agent and its coordinate to the grid:
            #self.grid.place_agent(a_fpb, (x,y))
            # add the agent to the schedule:
            self.schedule.add(a_fpb)
            agent_id += 1   
        
        # Farm Agent : Farm_Quality_Based (fqb)
        for n in range(self.n_fqb):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_fqb = Farm_Quality_Based(agent_id, self, (x,y), self.farm_profit_fqb, self.area_fqb)
            farms_indexes.append(a_fqb.unique_id)
            #self.grid.place_agent(a_fqb, (x,y))
            self.schedule.add(a_fqb)
            agent_id += 1 
        
        # Farm Agent : Farm_Sustainable_Organic_Based (fsob)
        for n in range(self.n_fsob):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_fsob = Farm_Sustainable_Organic_Based(agent_id, self, (x,y), self.farm_profit_fsob, self.area_fsob)
            farms_indexes.append(a_fsob.unique_id)
            #self.grid.place_agent(a_fsob, (x,y))
            self.schedule.add(a_fsob)
            agent_id += 1 
        
        # Customer Agent : Customer_Price_Sensitive (cps)
        for n in range(self.n_cps):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_cps = Customer_Price_Sensitive(agent_id, self, (x,y), farms_indexes, self.budget_min_cps, self.budget_max_cps)
            #self.grid.place_agent(a_cps, (x,y))
            self.schedule.add(a_cps)
            agent_id += 1
        
        # Customer Agent : Customer_Quality_Oriented (cqo)
        for n in range(self.n_cqo):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_cqo = Customer_Quality_Oriented(agent_id, self, (x,y), farms_indexes, self.budget_min_cqo, self.budget_max_cqo)
            #self.grid.place_agent(a_cqo, (x,y))
            self.schedule.add(a_cqo)
            agent_id += 1
        
        # Customer Agent : Customer_Environmental_Conscious (cec)
        for n in range(self.n_cec):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_cec = Customer_Environmental_Conscious(agent_id, self, (x,y), farms_indexes, self.budget_min_cec, self.budget_max_cec)
            #self.grid.place_agent(a_cec, (x,y))
            self.schedule.add(a_cec)
            agent_id += 1
            
        # Customer Agent : Customer_Dietary_Preference (cdp)
        for n in range(self.n_cdp):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            a_cdp = Customer_Dietary_Preference(agent_id, self, (x,y), farms_indexes, self.budget_min_cdp, self.budget_max_cdp)
            #self.grid.place_agent(a_cdp, (x,y))
            self.schedule.add(a_cdp)
            agent_id += 1
            
        # Data Collector:
        self.datacollector = mesa.DataCollector(model_reporters={"MoveAverageData":get_average_move_data,
                                                    "MoveCountData":ge_num_move_data,
                                                    "SatisfactionData":get_satisfaction_data,
                                                    "WeightNeedProducts": get_weight_need_products_data,
                                                    "AverageWeightNeedProducts":get_average_weight_need_products_data,
                                                    "WeightBoughtProducts": get_weight_bought_products_data,
                                                    "AverageWeightBoughtProducts":get_average_weight_bought_products_data,
                                                    "FarmBudgetData":get_farm_budget_data,
                                                    "FarmWasteData": get_farm_waste_data})

    def step(self):
        '''
        A step is the smallest unit of time in the model, and is often referred to as a tick.
        Every agent is expected to have a step method. The step method is the action the agent takes 
        when it is activated by the model schedule. 
        Here is each step consider as one week.
        '''
        farms_list = list()
        for farm in self.schedule.agents_by_type[Farm_Price_Based].values():
            farm.step()
            farms_list.append(farm)
        for farm in self.schedule.agents_by_type[Farm_Quality_Based].values():
            farm.step()
            farms_list.append(farm)
        for farm in self.schedule.agents_by_type[Farm_Sustainable_Organic_Based].values():
            farm.step(self.step_number)
            farms_list.append(farm)
        '''
        To pick an customer agent at random, we need a list of all customer agents. 
        '''
        a_cqo_shuffle = list(self.schedule.agents_by_type[Customer_Quality_Oriented].values())
        a_cps_shuffle = list(self.schedule.agents_by_type[Customer_Price_Sensitive].values())
        a_cec_shuffle = list(self.schedule.agents_by_type[Customer_Environmental_Conscious].values())
        a_cdp_shuffle = list(self.schedule.agents_by_type[Customer_Dietary_Preference].values())
        a_customers_shuffle = a_cqo_shuffle + a_cps_shuffle + a_cec_shuffle + a_cdp_shuffle
        # shuffle the order of the agents:
        self.random.shuffle(a_customers_shuffle)
        ''' activate and execute each agent’s step method: '''
        for agent in a_customers_shuffle:
            agent.step(farms_list)
        
        # collect data
        self.datacollector.collect(self)
        self.step_number += 1
        
    def run_model(self):
        '''Batch Runnig for the model'''
        for i in range(self.step_count):
            self.step()

# Result Class

In [ ]:
class Results(FEW):
    data_list = ['MoveAverageData', 'MoveCountData', 'SatisfactionData', 'WeightNeedProducts', 'AverageWeightNeedProducts',
       'WeightBoughtProducts', 'AverageWeightBoughtProducts', 'FarmBudgetData', 'FarmWasteData']
    def __init__(self, run_count: int):
        self.run_count = run_count
        self.data_frame_results_dict = dict.fromkeys(self.data_list, 0)
        
    def create_model(self):
        self.model = FEW(num_customer_price_sensitive = grid[1, 0].value, 
                    num_customer_quality_oriented = grid[1, 1].value, 
                    num_Customer_Environmental_Conscious = grid[1, 2].value,
                    num_Customer_Dietary_Preference = grid[1, 3].value, 
                    budget_min_customer_price_sensitive = grid[2, 0].value[0], 
                    budget_max_customer_price_sensitive = grid[2, 0].value[1],
                    budget_min_customer_quality_oriented = grid[2, 1].value[0],
                    budget_max_customer_quality_oriented = grid[2, 1].value[1],
                    budget_min_Customer_Environmental_Conscious = grid[2, 2].value[0], 
                    budget_max_Customer_Environmental_Conscious = grid[2, 2].value[1], 
                    budget_min_Customer_Dietary_Preference = grid[2, 3].value[0],
                    budget_max_Customer_Dietary_Preference = grid[2, 3].value[1], 
                    num_Farm_Price_Based = grid[3, 1].value, 
                    num_Farm_Quality_Based = grid[4, 1].value, 
                    num_Farm_Sustainable_Organic_Based = grid[5, 1].value,
                    area_Farm_Price_Based = grid[3, 2].value,
                    area_Farm_Quality_Based = grid[4, 2].value,
                    area_Farm_Sustainable_Organic_Based = grid[5, 2].value,
                    step_count = grid[3, 3].value,
                    width = 250, 
                    height = 250)
        return self.model
        
    def run(self):
        start_time = time.time()
        for run in range(self.run_count):
            self.model = self.create_model()
            self.model.run_model()
            self.create_data_frame()
        end_time = time.time()
        duration = end_time - start_time
        print("The simulation took", round(duration,1), "seconds.")
        print("Your simulation has done, now you can see the results.")

    def create_data_frame(self): 
        for key in self.data_list:
            data_array = self.model.datacollector.get_model_vars_dataframe()[key]
            self.data_frame_results_dict[key] += pd.DataFrame([data_array[i].values() for i in range(data_array.shape[0])], 
                                                            columns=data_array[0].keys())
        return self.data_frame_results_dict
    
    def create_integrated_data_frame(self):
        integrated_data_frame = pd.DataFrame()
        for i in self.data_list:
            for dataframe in self.data_frame_results_dict[i]:
                for column in dataframe.columns:
                    integrated_data_frame[column] = dataframe[column]

    def average_data(self, collected_data):
        '''this method devides the selected dataframe by number of runs:'''
        return self.data_frame_results_dict[collected_data] / self.run_count
    
    def table_customers_information(self):
        customer_population = [grid[1, 0].value, grid[1, 1].value, grid[1, 2].value, grid[1, 3].value]
 
        percentage = [round(i/sum(customer_population)*100,1) for i in customer_population]
        
        moves_num = [int(self.average_data('MoveCountData')[i].sum()) for i in self.average_data('MoveCountData').columns]
        
        moves_average = [round(self.average_data('MoveAverageData')[i].sum() / grid[3, 3].value,2) 
                         for i in self.average_data('MoveAverageData').columns]

        sum_weight_need = [round(self.average_data('WeightNeedProducts')[i].sum(),1)
                           for i in self.average_data('WeightNeedProducts').columns]
        
        average_weight_need = [round(self.average_data('AverageWeightNeedProducts')[i].sum() / grid[3, 3].value,1) 
                           for i in self.average_data('AverageWeightNeedProducts').columns]
        
        sum_weight_bought = [round(self.average_data('WeightBoughtProducts')[i].sum(),1) 
                           for i in self.average_data('WeightBoughtProducts').columns]
        
        average_weight_bought=[round(self.average_data('AverageWeightBoughtProducts')[i].sum() / grid[3, 3].value,1) 
                           for i in self.average_data('AverageWeightBoughtProducts').columns]
        
        satisfaction = [round(self.average_data('SatisfactionData')[i].sum() / grid[3, 3].value,1) 
                           for i in self.average_data('SatisfactionData').columns]

        dic_customers = {
                'Number' : customer_population,
                'Percentage' : percentage,
                "Number of Moves" : moves_num,
                "Average of Moves" : moves_average,
                "Sum Weight Needs" : sum_weight_need,
                "Average Weight Needs" : average_weight_need,
                "Sum Weight Bought" : sum_weight_bought,
                "Average Weight Bought" : average_weight_bought,
                "Satisfaction" : satisfaction
               }
        
        df_customers = pd.DataFrame(data=dic_customers, index=["Price Sensitive", "Quality Oriented", 
                                                        "Environmentally Conscious", "Dietary Preference-Based"])
        
        return df_customers
    
    def table_farms_information(self):
        
        obj_pro_fpb = Products(quality="Low", farming_method=["non-organic"],
                               vegetables=["Tomatoes", "Potatoes", "Carrots"], land=grid[3, 2].value)
        pro_fpb = sum(obj_pro_fpb.Cultivate().values()) * grid[3, 3].value
        water_fpb = sum(obj_pro_fpb.calculate_water().values()) * grid[3, 3].value
        energy_fpb = sum(obj_pro_fpb.calculate_energy().values()) * grid[3, 3].value
        
        obj_pro_fqb = Products(quality="High", farming_method=["organic"], 
                        vegetables=["Tomatoes", "Potatoes", "Carrots", "Eggplants", "Spinach"], land=grid[4, 2].value)
        pro_fqb = sum(obj_pro_fqb.Cultivate().values()) * grid[3, 3].value
        water_fqb = sum(obj_pro_fqb.calculate_water().values()) * grid[3, 3].value
        energy_fqb = sum(obj_pro_fqb.calculate_energy().values()) * grid[3, 3].value
        
        obj_pro_fsob = Products(quality="Average", farming_method=["organic", "sustainable"],
                        vegetables=["Tomatoes", "Potatoes","Carrots", "Eggplants", "Spinach"], land=grid[5, 2].value)
        pro_fsob = sum(obj_pro_fsob.Cultivate().values()) * grid[3, 3].value
        water_fsob = sum(obj_pro_fsob.calculate_water().values()) * grid[3, 3].value
        energy_fsob = sum(obj_pro_fsob.calculate_energy().values()) * grid[3, 3].value
        
        pro_lst = [pro_fpb, pro_fqb, pro_fsob]
        water_lst = [water_fpb, water_fqb, water_fsob]
        energy_lst = [energy_fpb, energy_fqb, energy_fsob]
        
        
        total_waste = [self.average_data('FarmWasteData')[i].sum() for i in self.average_data('FarmWasteData').columns]
        
        collected_water_fsob = sum([collect_rain_water(i, grid[5, 2].value) for i in range(grid[3, 3].value)])
        generated_energy_fsob = sum([generate_solar_energy(i) for i in range(grid[3, 3].value)])
        
        dic_farms = {
            'Farm Area (m²)' : [grid[3, 2].value, grid[4, 2].value, grid[5, 2].value],
            'Production Quality' : ['Low', 'High', 'Average'],
            'Production Method' : ['Non-Organic', 'organic', 'Organic & Sustainable'],
            'Total Production (kg)' : pro_lst,
            'Total Waste (kg)' : total_waste,
            'Waste Percentage (%)' : [int(total_waste[i]/pro_lst[i]*100) for i in range(len(total_waste))],
            'Profit (Euro)':[int(self.average_data('FarmBudgetData')[i][grid[3, 3].value-1]) 
                             for i in self.average_data('FarmBudgetData').columns],
            'Water Requirements (Liters)' : water_lst,
            'Percentage of Water Purchased (%)' : [100, 100, round((water_fsob-collected_water_fsob)/water_fsob*100,2)] ,
            'Energy Requirements (KWh)' : energy_lst,
            'Percentage of Energy Purchased (KWh)' : [100,100,int((energy_fsob-generated_energy_fsob)/energy_fsob*100)]
        }
        
        df_farms = pd.DataFrame(data=dic_farms, index=['Price-Based', 'Quality-Based', 'Eco-Environmentally Focused'])
        print()
#         print("Eco-Environmentally Focused Strategy Farm uses the rain water collection system (Green water print) with below specifications: ")
#         print("Catchment Area = ", catchment_area, 'm2')
#         print("Average Runoff Coefficient = ", runoff_coefficient)
#         print("depth of the rainwater harvesting system = ", depth, 'm')
#         print()
#         print("Eco-Environmentally Focused Strategy Farm uses the PV to grow more sustainable vegetables with below specifications: ")
#         print("model = ", pv_model)
#         print("Module Efficiency = ", eff)
#         print("PV Area = ", A, "m2")
#         print("PV Price = ", pv_price, "Euro")
#         print("Number of PV = ", PV_number)
#         print()
        return df_farms
    
    def pie_plot_farms_products(self):
        obj_pro_fpb = Products(quality="Low", farming_method=["non-organic"],
                               vegetables=["Tomatoes", "Potatoes", "Carrots"], land=grid[3, 2].value)
        value_obj_pro_fpb = [value * grid[3, 3].value for value in obj_pro_fpb.Cultivate().values()]
        key_obj_pro_fpb = [key for key in obj_pro_fpb.Cultivate().keys()]
        
        obj_pro_fqb = Products(quality="High", farming_method=["non-organic"], 
                        vegetables=["Tomatoes", "Potatoes", "Carrots", "Eggplants", "Spinach"], land=grid[4, 2].value)
        value_obj_pro_fqb = [value * grid[3, 3].value for value in obj_pro_fqb.Cultivate().values()]
        key_obj_pro_fqb = [key for key in obj_pro_fqb.Cultivate().keys()]
        
        obj_pro_fsob = Products(quality="Average", farming_method=["organic", "sustainable"],
                        vegetables=["Tomatoes", "Potatoes","Carrots", "Eggplants", "Spinach"], land=grid[5, 2].value)
        value_obj_pro_fsob = [value * grid[3, 3].value for value in obj_pro_fsob.Cultivate().values()]
        key_obj_pro_fsob = [key for key in obj_pro_fsob.Cultivate().keys()]
        
        c = ['Tomato', 'SandyBrown', 'OrangeRed', 'purple', 'SeaGreen']
        
        # Create subplots with adjusted figure size
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].pie(value_obj_pro_fpb, labels=key_obj_pro_fpb, colors=c[0:3], 
                                    autopct=lambda x: f'{int(x/100 * sum(value_obj_pro_fpb))}', textprops={'fontsize': 12})
        axes[0].set_title('Price-Based Farm Products', fontsize=15)
        
        axes[1].pie(value_obj_pro_fqb, labels=key_obj_pro_fqb, colors=c, 
                                    autopct=lambda x: f'{int(x/100 * sum(value_obj_pro_fqb))}', textprops={'fontsize': 12})
        axes[1].set_title('Quality-Based Farm Products', fontsize=15)
        
        axes[2].pie(value_obj_pro_fsob, labels=key_obj_pro_fsob, colors=c, 
                                    autopct=lambda x: f'{int(x/100 * sum(value_obj_pro_fsob))}', textprops={'fontsize': 12})
        axes[2].set_title('Eco-Environmentally Focused Farm Products', fontsize=15)

        # Display the figure
        plt.tight_layout()
        print()
        print()
        plt.show()
    
    def plot_information(self, collected_data):
        match collected_data:
            case 'MoveAverageData': 
                ylable = "Average Number of Move"
                title = "Average Number of Move for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'MoveCountData':
                ylable = "Total Number of Move"
                title = "Total Number of Move for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'SatisfactionData':
                ylable = "Customer Satisfaction (%)"
                title = "Customer Satisfaction percentage for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'WeightNeedProducts':
                ylable = "Total Weight Need Products (kg)"
                title = "Total Weight Need Products for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'AverageWeightNeedProducts':
                ylable = "Average Weight Need Products (kg)"
                title = "Average Weight Need Products for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'WeightBoughtProducts':
                ylable = "Total Weight Purchased Products (kg)"
                title = "Total Weight Purchased Products for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'AverageWeightBoughtProducts':
                ylable = "Average Weight Purchased Products (kg)"
                title = "Average Weight Purchased Products for each Customer agent type during simulation"
                color = ['Red', 'LightBlue', 'LightGreen', 'MediumPurple']
            case 'FarmBudgetData':
                ylable = "Farm Profit (Euro)"
                title = "Farm Profit for each Farm agent type during simulation"
                color = ['orange', 'gold', 'teal']
            case 'FarmWasteData':
                ylable = "Total Weight of Wasted Products (kg)"
                title = "Total Weight of Wasted Products for each Farm agent type during simulation"
                color = ['orange', 'gold', 'teal']
        return {'ylable':ylable, 'title':title, 'color':color}
                

    def plot(self, collected_data):
        df = self.average_data(collected_data)

        # Set the figure size
        plt.figure(figsize=(12, 6))

        # Set the x-ticks positions
        x = range(len(df))

        # Set the width of each bar
        bar_width = 0.25

        # Plot the line for each data in each column
        for i,column in enumerate(df.columns):
            plt.plot(x, df[column], label = re.sub(r"_.+", "", column), 
                     color=self.plot_information(collected_data)['color'][i], linestyle='-', linewidth=2)

        # Calculate the average for each row
        df['Average'] = df.mean(axis=1)

        # Plot the bar for the average of each column
        plt.bar(list(x), df['Average'], width=bar_width, color='Sienna', label='Average')

        # Set the x-tick labels
        plt.xticks(list(x) , range(1, len(df) + 1), fontsize=8)

        # Add labels and title
        plt.xlabel('Steps')
        plt.ylabel(self.plot_information(collected_data)['ylable'])
        plt.title(self.plot_information(collected_data)['title'])

        # Add a legend
        plt.legend()

        # Show the plot
        plt.show()
        plt.close()
        del df
        

# Widgets

In [ ]:
def create_widget_int_text(value, description):
    return widgets.IntText(
        value=value,
        description=description,
        disabled=False,
        layout=widgets.Layout(height='auto', width='200px'),
        style={'background-color':'red'}
    )

def create_normal_button():
    return widgets.Button(
    description='Run Simulation',
    disabled=False,
    style={'button_color':'Bisque'}, 
    tooltip='Run Simulation',
    layout=widgets.Layout(height='70px', width='auto')
)

def create_float_range_slider(min_val, max_val, step, value_range, color):
    return widgets.FloatRangeSlider(
                value=value_range,  # Initial range values
                min=min_val,  # Minimum value
                max=max_val,  # Maximum value
                step=step,  # Step size
                description='Budget',
                continuous_update=False,
                layout=widgets.Layout(width='auto'),
                style={'handle_color':color}
            )

def create_expanded_button(description):
    return widgets.Button(
    description=description,
    disabled=False,
    style={'button_color':'MediumAquaMarine', 'align-items': 'center', 'display': 'flex'},
    tooltip=description,
    layout=dict(width='99%', height='70px')
)

def create_widget_int_slider(min_val, max_val, step, value, color):
    return widgets.IntSlider(
        value=value,
        min=min_val,
        max=max_val,
        step=step,
        description='Area (m²)',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout=widgets.Layout(width='auto'),
        style={'handle_color':color}  
    )
    

def create_html(color, description):
    return widgets.HTML(
        value='<p style="background-color:{};text-align:center;font-size:95%;font-family:verdana;">{}</p>'
        .format(color, description),
        layout=widgets.Layout(height='auto', width='auto')
    )

grid = widgets.GridspecLayout(6, 4, height='600px')
grid[0, 0] = create_html('Red', 'Price-sensitive Customers')
grid[0, 1] = create_html('LightBlue', 'Quality-oriented Customers')
grid[0, 2] = create_html('LightGreen', 'Environmentally Conscious Customers')
grid[0, 3] = create_html('MediumPurple', 'Dietary Preference-based Customers')

grid[1, 0] = create_widget_int_text(20, 'Number')
grid[1, 1] = create_widget_int_text(20, 'Number')
grid[1, 2] = create_widget_int_text(5, 'Number')
grid[1, 3] = create_widget_int_text(5, 'Number')

grid[2, 0] = create_float_range_slider(0, 15, 0.2, [3,8],'blue')
grid[2, 1] = create_float_range_slider(0, 15, 0.2, [5,12],'blue')
grid[2, 2] = create_float_range_slider(0, 15, 0.2, [5,12],'blue')
grid[2, 3] = create_float_range_slider(0, 15, 0.2, [5,12],'blue')


grid[3, 0] = create_html('orange', 'Price-based Farm')
grid[3, 1] = create_widget_int_text(1, 'Number')
grid[3, 2] = create_widget_int_slider(0,2000,50,400,'DarkGray') 
grid[3, 3] = create_widget_int_text(52, 'Step Number')

grid[4, 0] = create_html('gold', 'Quality-based Farm')
grid[4, 1] = create_widget_int_text(1, 'Number')
grid[4, 2] = create_widget_int_slider(0,2000,50,400,'DarkGray')
grid[4, 3] = create_widget_int_text(5, 'Run Number')

grid[5, 0] = create_html('teal', 'Eco-environmentally Focused Farm')
grid[5, 1] = create_widget_int_text(1, 'Number')
grid[5, 2] = create_widget_int_slider(0,2000,50,200,'DarkGray')
grid[5, 3] = create_normal_button()

output_1 = widgets.Output()
output_2 = widgets.Output()
output_3 = widgets.Output()
output_4 = widgets.Output()
output_5 = widgets.Output()
output_6 = widgets.Output()
output_7 = widgets.Output()
output_8 = widgets.Output()
output_9 = widgets.Output()
output_10 = widgets.Output()
output_11 = widgets.Output()

button_interface = create_expanded_button("Show Interface")
button_table_farms_information = create_expanded_button("Show Farms Information Table")
button_farms_budget = create_expanded_button("Show Farms Profit Graph")
button_farms_waste = create_expanded_button("Show Farms Waste Graph")
button_table_customers_information = create_expanded_button("Show Customer Information Table")
button_weight_bought = create_expanded_button("Show Weight of Bought Products Graph")
button_average_weight_bought = create_expanded_button("Show the Average Weight of Bought Products Graph")
button_weight_need = create_expanded_button("Show Weight of Need's Customers Products Graph")
button_average_weight_need = create_expanded_button("Show the Average weight of Need's Customers Products Graph")
button_satisfaction = create_expanded_button("Show Customer's Satisfaction Graph")
button_move = create_expanded_button("Show Customer's move number Graph")
button_average_move = create_expanded_button("Show Customer's Average move Graph")

def run_button_click(b):
    simulation = Results(grid[4, 3].value)
    simulation.run()
    with output_1:
        display(simulation.table_farms_information())
        simulation.pie_plot_farms_products()
    with output_2:
        simulation.plot('FarmBudgetData')
    with output_3:
        simulation.plot('FarmWasteData')
    with output_4:
        display(simulation.table_customers_information())
    with output_5:
        simulation.plot('WeightBoughtProducts')
    with output_6:
        simulation.plot('AverageWeightBoughtProducts')
    with output_7:
        simulation.plot('WeightNeedProducts')
    with output_8:
        simulation.plot('AverageWeightNeedProducts')
    with output_9:
        simulation.plot('SatisfactionData')
    with output_10:
        simulation.plot('MoveCountData')
    with output_11:
        simulation.plot('MoveAverageData')
        
def click_button_interface(c):
    display(grid)
        
def click_button_table_farms_information(b):
    display(output_1)
    
def click_button_farms_budget(b):
    display(output_2)
    
def click_button_farms_waste(b):
    display(output_3)
    
def click_button_table_customers_information(b):
    display(output_4)
    
def click_button_weight_bought(b):
    display(output_5)
    
def click_button_average_weight_bought(b):
    display(output_6)
    
def click_button_weight_need(b):
    display(output_7)
    
def click_button_average_weight_need(b):
    display(output_8)
    
def click_button_satisfaction(b):
    display(output_9)
    
def click_button_move(b):
    display(output_10)
    
def click_button_average_move(b):
    display(output_11)


grid[5, 3].on_click(run_button_click)
button_interface.on_click(click_button_interface)
button_table_farms_information.on_click(click_button_table_farms_information)
button_farms_budget.on_click(click_button_farms_budget)
button_farms_waste.on_click(click_button_farms_waste)
button_table_customers_information.on_click(click_button_table_customers_information)
button_weight_bought.on_click(click_button_weight_bought)
button_average_weight_bought.on_click(click_button_average_weight_bought)
button_weight_need.on_click(click_button_weight_need)
button_average_weight_need.on_click(click_button_average_weight_need)
button_satisfaction.on_click(click_button_satisfaction)
button_move.on_click(click_button_move)
button_average_move.on_click(click_button_average_move)